## Introduction
$\def\lex{\lambda_{ex}}$
The quantum yield of a fluorophore is defined as the number of photons the material emits for every photon that it absorbed. That is, $\Phi\equiv \dfrac{N_{em}}{N_{ab}}$. To calculate these two $N$ values, we use a Photon Technologies Internaltional fluorescence spectrometer system (PTI) to measure two emission spectra at some common excitation wavelength $\lambda_{ex}$. The first is a blank spectrum, $I_b(\lambda)$, and is taken using a cuvette filled exclusively with our solvent of choice, e.g. ethanol (EtOH), LAB. The second spectrum is taken with concentration of our fluorophore in the solvent. We label the raw emission spectrum with $I_f(\lambda)$, for fluorophore. 

These two samples must be corrected to compensate for:
1. Time-dependent variations in the PTI light source intensity (a Xenon arc lamp)
2. Wavelegnth-dependent sensitivities in the system, espectially those from the integrating sphere (IS)

Light from the arc lamp is immediately directed into a series of monochromators that only allows light to come through from a small range of wavelengths around $\lex$. The remaining light then passes through a beamsplitter. One beam goes to cuvette samples, while the other is directed to a photodiode. The resulting potential difference is measured and used to determine how the incoming intensity of light varies in time. 

In [28]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

import PTI.Corrections as PTICorr
from PTI.ReadDataFiles import PTIData
import PTI.BaselineFitting as PTIBaseline
import PTI.QuantumYield as PTIQY

## Import the Data

In [20]:
# The file paths for the blank LAB measurements
LAB_paths = ["Henry/Sphere/bisMSB_LAB/EmissionScan_LAB_ex350_2sec_160823.txt",
             "Henry/Sphere/bisMSB_LAB/EmissionScan_LAB_ex360_2sec_160823.txt",
             "Henry/Sphere/bisMSB_LAB/EmissionScan_LAB_ex370_2sec_160823.txt",
             "Henry/Sphere/bisMSB_LAB/EmissionScan_LAB_ex380_2sec_160823.txt"]

# The file paths for the 4.47 mg/L bisMSB in LAB measurements
bisMSB_4x47_paths = ["Henry/Sphere/bisMSB_LAB/EmissionScan_bisMSBinLAB_4.47mgL_ex350_2sec_160824.txt",
                     "Henry/Sphere/bisMSB_LAB/EmissionScan_bisMSBinLAB_4.47mgL_ex360_2sec_160824.txt",
                     "Henry/Sphere/bisMSB_LAB/EmissionScan_bisMSBinLAB_4.47mgL_ex370_2sec_160824.txt",
                     "Henry/Sphere/bisMSB_LAB/EmissionScan_bisMSBinLAB_4.47mgL_ex380_2sec_160824.txt"]

# The file paths for the blank ethanol measurements
EtOH_paths = ["Henry/Sphere/PPO_ETOH/EmissionScan_ETOH_ex310_2sec_160830.txt",
              "Henry/Sphere/PPO_ETOH/EmissionScan_ETOH_ex320_2sec_160830.txt",
              "Henry/Sphere/PPO_ETOH/EmissionScan_ETOH_ex330_2sec_160830.txt",
              "Henry/Sphere/PPO_ETOH/EmissionScan_ETOH_ex340_2sec_160830.txt"]

# The file paths for the 0.31 mg/L PPO in ethanol measurements
PPO_0x31_paths = ["Henry/Sphere/PPO_ETOH/EmissionScan_0x31gperL_PPOinETOH_ex310_2sec_160831.txt",
                  "Henry/Sphere/PPO_ETOH/EmissionScan_0x31gperL_PPOinETOH_ex320_2sec_160831.txt",
                  "Henry/Sphere/PPO_ETOH/EmissionScan_0x31gperL_PPOinETOH_ex330_2sec_160831.txt",
                  "Henry/Sphere/PPO_ETOH/EmissionScan_0x31gperL_PPOinETOH_ex340_2sec_160831.txt"]

# The file paths for the 3.14 mg/L PPO in ethanol measurements
PPO_3x14_paths = ["Henry/Sphere/PPO_ETOH/EmissionScan_3x14gperL_PPOinETOH_ex310_2sec_160831.txt",
                  "Henry/Sphere/PPO_ETOH/EmissionScan_3x14gperL_PPOinETOH_ex320_2sec_160831.txt",
                  "Henry/Sphere/PPO_ETOH/EmissionScan_3x14gperL_PPOinETOH_ex330_2sec_160831.txt",
                  "Henry/Sphere/PPO_ETOH/EmissionScan_3x14gperL_PPOinETOH_ex340_2sec_160831.txt"]

In [21]:
'''Requires: A list of file paths for PTI data files
   Effect:   Takes each path and uses it to read into a PTIData instance
   Returns:  A list of PTIData instances. Each element comes from the corresponding
             path in the list argument.'''
def convert_paths_to_PTIData_objs(list_of_paths):
    list_of_PTIData = list()
    for path in list_of_paths:
        list_of_PTIData.append(PTIData(path))
    return list_of_PTIData

In [22]:
LAB = convert_paths_to_PTIData_objs(LAB_paths)
bisMSB_4x47 = convert_paths_to_PTIData_objs(bisMSB_4x47_paths)

ETOH = convert_paths_to_PTIData_objs(EtOH_paths)
PPO_0x31 = convert_paths_to_PTIData_objs(PPO_0x31_paths)
PPO_3x14 = convert_paths_to_PTIData_objs(PPO_3x14_paths)

## Undoing the Real-time Corrections made by the Felix GX Software

In [5]:
'''Arrays to store calculations'''
# The excitation wavelengths of each scan
ex_wavelengths = list()

# The mean decorrection:raw ratio
mean_ratios = list()

# The average diode signal
mean_diode_signal = list()

'''Decorrection ata and parameters'''
# Collect all the data instances into one cumulative list
all_data_sets = LAB + bisMSB_4x47 + ETOH + PPO_0x31 + PPO_3x14

# Which corrections to undo
diode_signal = True # We will use the stored diode signal to undo this correction
ex_LUT = False
em_LUT = False

# The degree of spline interpolation to use for look-up table values
interp_degree = 'cubic' 

'''Decorrecting and creating plots'''
for data_set in all_data_sets:
    fname = data_set.file_path.split('/')[-1][:-4]
    # Get the decorrected spectrum
    decorr = PTICorr.decorrect_cor_data(PTIData_instance = data_set,
                                        interp_method = interp_degree,
                                        diode = diode_signal,
                                        excorr = ex_LUT,
                                        emcorr = em_LUT)
    
    
    
    # Compare the decorrected to the raw data
    difference = decorr.cor_data - data_set.raw_data
    ratio = decorr.cor_data / data_set.raw_data
    
    # Fill the two auxilliary arrays
    ex_wavelengths.append(data_set.ex_range[0])
    mean_ratios.append(np.mean(ratio))
    mean_diode_signal.append(np.mean(data_set.diode))
    
    # Create our figure and 3 subplots
    fig = plt.figure(figsize = (16,10))
    ax1 = plt.subplot2grid((2,2), (0,0), colspan = 2)
    ax2 = plt.subplot2grid((2,2), (1,0))
    ax3 = plt.subplot2grid((2,2), (1,1))
    
    # Adding titles to everything
    suptitle = fig.suptitle("Effects of Manually Decorrecting from Felix\n%s" %fname,y = 1.06, fontsize = 20)
    ax1.set_title("Decorrected and Raw Data Data", fontsize = 15)
    ax2.set_title("Difference between Decorrected and Raw Data", fontsize = 15)
    ax3.set_title("Ratio between Decorrected and Raw Data\n" + 
                  r'$\mu =$ %.2f' %(np.mean(ratio)) + '\t' + 
                  r'$\sigma =$ %.2f $\cdot 10^{-8}$' %(1.0e8*np.std(ratio)), fontsize = 15)
    
    # Plot the relevant data in each subplot
    ax1.plot(data_set.wavelengths, data_set.raw_data, 'k')
    ax1.plot(decorr.wavelengths, decorr.cor_data, 'r')
    ax2.plot(data_set.wavelengths, difference)
    ax3.plot(data_set.wavelengths, ratio)
    
    # Format the subplots
    ax1.legend(["Raw", "Decorrected"], fontsize = 15)
    
    
    ax3.set_ylim([0.5, 1.5])
    
    for ax in [ax1, ax2, ax3]:
        ax.grid()
        ax.set_xlim([300, 650])
        
    plt.tight_layout()
    plt.savefig("2016_Sphere_Decorrection_Plots/%s.png" %(fname),
               dpi=fig.dpi, bbox_inches='tight',bbox_extra_artists=[suptitle])
    plt.close('all')


In [6]:
fig = plt.figure(figsize = (16,10))
ax = fig.add_subplot(1,1,1)
ax.plot(mean_ratios, ex_wavelengths, 'go')
ax.plot(mean_ratios, mean_diode_signal, 'bo')
ax.set_yscale('log')
plt.show()